In [ ]:
#import stuff

%load_ext autoreload
%autoreload 2

import sys
#change path to wherever you put the code
sys.path.insert(0, '/Users/alexandrasockell/Organoids/Code/ProcessingPack_Organoids/processingpack')

from copy import deepcopy
from pathlib import Path

import numpy as np
import skimage
from skimage import measure
import pandas as pd
import matplotlib.pyplot as pl

import ast
import operator

import chip
import experiment as exp
import chipcollections as collections
from org_extension import *

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
#define square finder
import sys
PY3 = sys.version_info[0] == 3

# Python 2/3 compatibility
if PY3:
    xrange = range

import numpy as np
from scipy import spatial as sp
import cv2 as cv
from matplotlib import pyplot as plt

def angle_cos(p0, p1, p2):
    d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
    return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )

def find_squares(img):
    img = cv.GaussianBlur(img, (25, 25), 0)
    squares = []
    for gray in cv.split(img):
        for thrs in xrange(0, 255, 26):
            if thrs == 0:
                bin = cv.Canny(gray, 0, 50, apertureSize=5)
                bin = cv.dilate(bin, None)
            else:
                _retval, bin = cv.threshold(gray, thrs, 255, cv.THRESH_BINARY)
            bin, contours, _hierarchy = cv.findContours(bin, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
            for cnt in contours:
                cnt_len = cv.arcLength(cnt, True)
                cnt = cv.approxPolyDP(cnt, 0.02*cnt_len, True)
                if len(cnt) == 4 and cv.contourArea(cnt) > 1000 and cv.isContourConvex(cnt):
                    cnt = cnt.reshape(-1, 2)
                    max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in xrange(4)])
                    if max_cos < 0.1:
                        squares.append(cnt)
    return squares

In [ ]:
#define html table writers for summarizing data

def html_header(output_html):
    with open(output_html, "w") as out:
        header = '''
        <html>
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">
        <link rel="stylesheet" href="https://cdn.datatables.net/1.10.19/css/jquery.dataTables.min.css">
        <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>
        <script src="https://cdn.datatables.net/1.10.19/js/jquery.dataTables.min.js"></script>
        <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/js/bootstrap.min.js"></script>

        <script>
        $(document).ready(function () {
        $('#microwell-data').DataTable();
        });
        </script>

        <table id="microwell-data" class="table table-striped table-bordered table-sm" cellspacing="0" width="100%">
        <thead>
        <tr>
        <th>well_id</th><th>day</th><th>median pixel intensity</th><th>hyst1 area</th><th>hyst2 area</th>
        <th>well</th><th>mw_area</th><th>hyst1</th><th>hyst2</th>
        </tr>
        </thead>
        <tbody>
        '''
        out.write(header)
        
def html_row(well,day,median,sum1,sum2,output_html):
    with open(output_html, "a") as output:
        row = '''
        <tr>
            <td>well{0}</td><td>{1}</td><td>{2}</td><td>{3}</td><td>{4}</td>
            <td><img src = ./microwells/well{0:04d}_day{1:02d}_well.png alt = well{0:04d}_day{1:02d}_well.png></td>
            <td><img src = ./microwells/well{0:04d}_day{1:02d}_mw_area.png alt = well{0:04d}_day{1:02d}_mw_area.png></td>
            <td><img src = ./microwells/well{0:04d}_day{1:02d}_hyst1.png alt = well{0:04d}_day{1:02d}_hyst1.png></td>
            <td><img src = ./microwells/well{0:04d}_day{1:02d}_hyst2.png alt = well{0:04d}_day{1:02d}_hyst2.png></td>

        </tr>
        '''.format(well,day,median,sum1,sum2)
        output.write(row)
        
        
def html_footer(output_html):
    with open(output_html, "a") as file:
        footer = '''
        </tbody></table>
        </html>
        '''
        file.write(footer)

In [ ]:
#set corner positions (from stitched image, find largest rectangle of fully enclosed subgrids, see example)
# format is oc[day] = ((top left), (top right), (bottom left), (bottom right))

oc = {}

oc[0] = ((1043,3839), (12821,5267), (224,11562), (11953,12910))
oc[1] = ((1070,3771), (12846,5204), (246,11494), (11976,12844))
oc[2] = ((979,3538), (12759,4939), (175,11256), (11900,12591))
oc[3] = ((982,3545), (12761,4949), (175,11266), (11900,12596))
oc[4] = ((975,3547), (12753,4948), (165,11269), (11895,12602))
oc[5] = ((984,3549), (12763,4950), (176,11271), (11901,12600))
oc[6] = ((978,3542), (12759,4953), (168,11269), (11901,12602))
oc[7] = ((1222,3700), (12995,5138), (397,11415), (12135,12776))
oc[8] = ((1223,3694), (12999,5134), (398,11419), (12135,12771))
oc[9] = #etc
oc[10] = 
oc[11] = 
oc[12] = 
oc[13] = 
oc[14] = 
oc[15] = 
oc[16] = 
oc[17] = 
oc[18] = 
oc[19] = 
oc[20] = #this is the trypan image

In [ ]:
#rotate images, then save .tif and .csv files of subgrids
from datetime import datetime

subarray_dims = (5,3) #number of col,row in the fully enclosed rectangle you selected (each subgrid counts as one, so in the example image this would be 5col,3row)

#these can stay the same
tile_dims = (20, 20) 
channel = '2bf'
exposure = 10
date = datetime(2019, 5, 7)
well = 1 

timecourse_length = 21 #0 indexed
for day in range(timecourse_length):
    #change paths to directory containing stitched images and/or code
    srcHandle = Path('/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high/day{0}_stitched.tif'.format(day))
    targetHandle = Path('/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high/day{0}_stitched_rotated.tif'.format(day))
    rot_img = readAndRotateImg(srcHandle, oc[day, subarray_dims, targetHandle = targetHandle)
    rot_corners = transformCorners(readTiff(srcHandle), rot_img, oc[day])
    divisions = getPartitions(rot_corners, subarray_dims)
    root = '/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high/'
    description = 'Day{0}'.format(day)
    operator = 'AS'
    chip.ChipImage.stampWidth = 194
    e = exp.Experiment(description, root, operator) 
    pinlist_path = '/Users/alexandrasockell/Organoids/Code/ProcessingPack_Organoids/Pinlist/20x20_pinlist.csv'
    pinlist = e.read_pinlist(pinlist_path)
    arrayRepo = processTiles(str(targetHandle), e, pinlist, divisions, tile_dims, subarray_dims, channel, exposure, date = date, well = well, desc = e.info)
    outRoot = Path('/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high')

    ar = arrayRepo[0, 0] 
    hashStr = generateWellIdentifier(ar)
    summary_imgPath = outRoot.joinpath('{}.tif'.format(hashStr))
    summary_csvPath = outRoot.joinpath('{}.csv'.format(hashStr))
    
    summaryDF = summarizeArrayRepo(arrayRepo)
    summaryDF.to_csv(summary_csvPath)
    writeSubArraySummaryImg(arrayRepo, summary_imgPath)

In [ ]:
#separate subgrids into individual microwell images and save images, csv file, and html file
#also does some processing of the microwell images to calculate area of the cell/organoid within, you may or may not need this

from skimage import data, filters
import operator
from mpl_toolkits.axes_grid1 import make_axes_locatable

#also saves csv and html files with values for each day
output_csv = '/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high/well_summary.csv'
output_html = '/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high/well_summary.html'
html_header(output_html)
with open(output_csv, "w") as out:
    header = 'well id' + ',' + 'day' + ',' + 'median pixel intensity' + ',' + 'hyst1 area' + ',' + 'hyst2 area' + "\n"
    out.write(header)   

#iterate through days
for day in range(timecourse_length):
    #import csv and tif files you just saved above
    reimport = pd.read_csv('/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high/Day{0}-1-20190507.csv'.format(day))
    imagePath = '/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high/Day{0}-1-20190507.tif'.format(day)
    with skimage.external.tifffile.TiffFile(imagePath) as tif:
        data = tif.asarray()

    #iterate through wells
    for i in range(len(reimport)):
        #pull out well location from csv
        chamberInfo = reimport.iloc[i]
        #slice image to that well
        xslice = ast.literal_eval(chamberInfo.summaryImg_xslice)
        yslice = ast.literal_eval(chamberInfo.summaryImg_yslice)
        well = data[chamberInfo.stack_indexer][xslice[0]:xslice[1],yslice[0]:yslice[1]]
        pl.imsave('/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high/microwells/well{0:04d}_day{1:02d}_well.png'.format(i, day), well, cmap="gray")
        
        #use the square finder to find the inside of the well 
        squares = find_squares(well)
        squares = np.sort(np.array(squares))
        
        #find square with smallest perimeter and set that equal to inner mw_area
        perimeters = {}
        for x in range(len(squares)):
            perimeter = cv.arcLength(squares[x], True)
            perimeters[x] = perimeter
        if len(perimeters) == 0:
            print('perimeters not found')
        else:
            small_key = min(perimeters.items(), key=operator.itemgetter(1))[0]
            min_x = np.min(squares[small_key][:,0])
            max_x = np.max(squares[small_key][:,0])
            min_y = np.min(squares[small_key][:,1])
            max_y = np.max(squares[small_key][:,1])

            mw_area = well[min_x+5:max_x-5,min_y+5:max_y-5]
            pl.imsave('/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high/microwells/well{0:04d}_day{1:02d}_mw_area.png'.format(i, day), mw_area, cmap="gray")

            #normalize image by median pixel intensity
            filtermedian = np.abs(mw_area.astype(float) - np.median(mw_area))

            kernel =  cv.getStructuringElement(cv.MORPH_ELLIPSE,(7,7))
            closing = cv.morphologyEx(filtermedian, cv.MORPH_CLOSE, kernel)
            hyst1 = filters.apply_hysteresis_threshold(closing, 15,10 )
            pl.imsave('/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high/microwells/well{0:04d}_day{1:02d}_hyst1.png'.format(i, day), hyst1.astype(float), cmap="gray")

            kernel =  cv.getStructuringElement(cv.MORPH_ELLIPSE,(10,10))
            opening = cv.morphologyEx(closing, cv.MORPH_OPEN, kernel)

            hyst2 = filters.apply_hysteresis_threshold(opening, 15,10 )
            pl.imsave('/Users/alexandrasockell/Organoids/Data/stitched/PDAC_drugtest/Gem_high/microwells/well{0:04d}_day{1:02d}_hyst2.png'.format(i, day), hyst2.astype(float), cmap="gray")

            with open(output_csv, "a") as output:
                line = str(i) + ',' + str(day) + ',' + str(np.median(mw_area)) + ',' + str(np.sum(hyst1)) + ',' + str(np.sum(hyst2)) + "\n"
                output.write(line)
            
            html_row(i,day,np.median(mw_area),np.sum(hyst1),np.sum(hyst2),output_html)


            print('well' + " " +  str(i) + " " + 'day' + " " + str(day))

html_footer(output_html)